In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn
import pymysql
import pickle

In [2]:
con = pymysql.connect(host="127.0.0.1", db="riot", user="root", password="root1234", charset='utf8')
addata = pd.read_sql("select * from addata", con)
addata

D:\kdigital\anaconda\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,ano,blue_ad_champ,bad_kill,bad_gold,bad_damageDone,bad_damageTaken,bad_level,bad_minion,red_ad_champ,rad_kill,rad_gold,rad_damageDone,rad_damageTaken,rad_level,rad_minion,result
0,1,Kaisa,0,3392,2248,5498,8,60,Lucian,6,7481,9243,7186,9,126,1
1,2,Jinx,2,6040,6204,7615,9,90,Tristana,7,8100,10010,8701,9,97,0
2,3,MissFortune,6,6926,9856,4673,9,116,Ashe,1,4636,2934,6091,8,104,0
3,4,Nilah,3,6395,4461,6737,10,127,Ezreal,1,5658,6200,4227,9,121,1
4,5,Sivir,1,4726,3406,5700,8,106,Kaisa,3,7225,4443,6111,10,135,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29543,29545,Jinx,2,5800,5447,3734,8,111,Kaisa,3,5974,6635,5672,9,114,1
29544,29546,Jinx,4,7059,7552,4142,10,135,Jhin,2,5469,4393,5890,9,100,0
29545,29547,Kaisa,5,6381,7306,7930,8,90,Yasuo,1,6324,5912,8809,10,105,1
29546,29548,Samira,3,5779,7806,10666,9,83,Caitlyn,2,5355,8833,5921,8,83,1


In [3]:
# 데이터 타입 확인
addata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29548 entries, 0 to 29547
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   ano              29548 non-null  int64 
 1   blue_ad_champ    29548 non-null  object
 2   bad_kill         29548 non-null  int64 
 3   bad_gold         29548 non-null  int64 
 4   bad_damageDone   29548 non-null  int64 
 5   bad_damageTaken  29548 non-null  int64 
 6   bad_level        29548 non-null  int64 
 7   bad_minion       29548 non-null  int64 
 8   red_ad_champ     29548 non-null  object
 9   rad_kill         29548 non-null  int64 
 10  rad_gold         29548 non-null  int64 
 11  rad_damageDone   29548 non-null  int64 
 12  rad_damageTaken  29548 non-null  int64 
 13  rad_level        29548 non-null  int64 
 14  rad_minion       29548 non-null  int64 
 15  result           29548 non-null  int64 
dtypes: int64(14), object(2)
memory usage: 3.6+ MB


In [4]:
# champ code 불러오기
with open('champ.pickle','rb') as f:
    champCode = pickle.load(f)

champCode[['champion_key','en_name']]

,champion_key,en_name
0,266,Aatrox
1,103,Ahri
2,84,Akali
3,166,Akshan
4,12,Alistar
...,...,...
156,221,Zeri
157,115,Ziggs
158,26,Zilean
159,142,Zoe


In [5]:
# 챔피언명 라벨링
def changeCode(x):
    for code, name in champCode[['champion_key','en_name']].iterrows():
        if x == name[1]:
            return name[0]
red_ad_champ_code = addata['red_ad_champ'].apply(changeCode)
blue_ad_champ_code = addata['blue_ad_champ'].apply(changeCode)

chmapToCode = pd.concat([red_ad_champ_code,blue_ad_champ_code],axis=1)
chmapToCode

,red_ad_champ,blue_ad_champ
0,236,145
1,18,222
2,22,21
3,81,895
4,145,15
...,...,...
29543,145,222
29544,202,222
29545,157,145
29546,51,360


In [6]:
#라벨링된 데이터로변경
labelAd = pd.concat([chmapToCode,addata.drop(['blue_ad_champ','red_ad_champ'], axis=1)],axis=1)

#결측치 제거
labelAd = labelAd.dropna(axis=0)

#타입변경
labelAd = labelAd.astype({'red_ad_champ':'int64'})
labelAd.info()
labelAd

<class 'pandas.core.frame.DataFrame'>
Int64Index: 29539 entries, 0 to 29547
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   red_ad_champ     29539 non-null  int64 
 1   blue_ad_champ    29539 non-null  object
 2   ano              29539 non-null  int64 
 3   bad_kill         29539 non-null  int64 
 4   bad_gold         29539 non-null  int64 
 5   bad_damageDone   29539 non-null  int64 
 6   bad_damageTaken  29539 non-null  int64 
 7   bad_level        29539 non-null  int64 
 8   bad_minion       29539 non-null  int64 
 9   rad_kill         29539 non-null  int64 
 10  rad_gold         29539 non-null  int64 
 11  rad_damageDone   29539 non-null  int64 
 12  rad_damageTaken  29539 non-null  int64 
 13  rad_level        29539 non-null  int64 
 14  rad_minion       29539 non-null  int64 
 15  result           29539 non-null  int64 
dtypes: int64(15), object(1)
memory usage: 3.8+ MB


,red_ad_champ,blue_ad_champ,ano,bad_kill,bad_gold,bad_damageDone,bad_damageTaken,bad_level,bad_minion,rad_kill,rad_gold,rad_damageDone,rad_damageTaken,rad_level,rad_minion,result
0,236,145,1,0,3392,2248,5498,8,60,6,7481,9243,7186,9,126,1
1,18,222,2,2,6040,6204,7615,9,90,7,8100,10010,8701,9,97,0
2,22,21,3,6,6926,9856,4673,9,116,1,4636,2934,6091,8,104,0
3,81,895,4,3,6395,4461,6737,10,127,1,5658,6200,4227,9,121,1
4,145,15,5,1,4726,3406,5700,8,106,3,7225,4443,6111,10,135,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29543,145,222,29545,2,5800,5447,3734,8,111,3,5974,6635,5672,9,114,1
29544,202,222,29546,4,7059,7552,4142,10,135,2,5469,4393,5890,9,100,0
29545,157,145,29547,5,6381,7306,7930,8,90,1,6324,5912,8809,10,105,1
29546,51,360,29548,3,5779,7806,10666,9,83,2,5355,8833,5921,8,83,1


In [7]:
addata = labelAd.drop('ano',axis=1)
addata

,red_ad_champ,blue_ad_champ,bad_kill,bad_gold,bad_damageDone,bad_damageTaken,bad_level,bad_minion,rad_kill,rad_gold,rad_damageDone,rad_damageTaken,rad_level,rad_minion,result
0,236,145,0,3392,2248,5498,8,60,6,7481,9243,7186,9,126,1
1,18,222,2,6040,6204,7615,9,90,7,8100,10010,8701,9,97,0
2,22,21,6,6926,9856,4673,9,116,1,4636,2934,6091,8,104,0
3,81,895,3,6395,4461,6737,10,127,1,5658,6200,4227,9,121,1
4,145,15,1,4726,3406,5700,8,106,3,7225,4443,6111,10,135,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29543,145,222,2,5800,5447,3734,8,111,3,5974,6635,5672,9,114,1
29544,202,222,4,7059,7552,4142,10,135,2,5469,4393,5890,9,100,0
29545,157,145,5,6381,7306,7930,8,90,1,6324,5912,8809,10,105,1
29546,51,360,3,5779,7806,10666,9,83,2,5355,8833,5921,8,83,1


## 데이터분석

In [8]:
from sklearn.model_selection import train_test_split

In [32]:
x_train, x_test, y_train, y_test=train_test_split(addata.drop('result',axis=1), addata['result'], test_size=0.27)
len(x_train), len(x_test), len(y_train), len(y_test)

(21563, 7976, 21563, 7976)

In [46]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='liblinear', random_state=1, C=0.05)
model.fit(x_train, y_train)
pred = model.predict(x_test)
(pred==y_test).mean()


0.691073219658977

In [47]:
from sklearn.metrics import accuracy_score, precision_score, recall_score
accuracy_score(pred,y_test),precision_score(pred,y_test),recall_score(pred,y_test)

(0.691073219658977, 0.697405599794503, 0.6785803549112722)